In [41]:
import ee
import os
import glob
import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

ee.Authenticate()

# Authenticate & initialize Earth Engine
ee.Initialize(project='biodiversity-478015')

In [42]:
# -------------------------
# CONFIG
# -------------------------
aoifile = '/mnt/e/gis/BaseData/MAV_Boundary_4326_wkb.parquet'
scale = 100  # Adjust based on your imagery resolution
aoi_gdf = gpd.read_parquet(aoifile)
aoi_geom = ee.Geometry.Polygon(list(aoi_gdf.geometry.union_all().exterior.coords))

In [57]:
def gdf_to_ee_geometry(gdf):
    if gdf.empty:
        raise ValueError("GeoDataFrame is empty.")

    # Merge all geometries into one
    geom = gdf.union_all()
    if geom.is_empty:
        raise ValueError("Geometry is empty.")

    # Convert to GeoJSON and then EE Geometry
    geom_json = geom.__geo_interface__
    return ee.Geometry(geom_json)

def get_dw_mode_image(months, year):
    start_month = min(months)
    end_month = max(months)

    start_date = ee.Date.fromYMD(year, start_month, 1)
    end_date = ee.Date.fromYMD(year, end_month, 1).advance(1, 'month')

    dw_collection = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").select("label")
    dw_filtered = dw_collection.filterDate(start_date, end_date)

    return dw_filtered.reduce(ee.Reducer.mode())


def compute_dw_percent_cover(dw_img, radius_m):
    class_ids = list(range(9))
    kernel = ee.Kernel.circle(radius=radius_m, units='meters', normalize=True)
    cover_images = []
    for class_id in class_ids:
        mask = dw_img.eq(class_id)
        pct = mask.reduceNeighborhood(ee.Reducer.mean(), kernel).multiply(100).rename(f'dw_class_{class_id}_pct_{radius_m}m')
        cover_images.append(pct)
    return ee.Image.cat(cover_images)


def compute_forest_metrics(dw_img, radius_m):
    forest_mask = dw_img.eq(1)
    non_forest_mask = dw_img.neq(1)

    forest_edges = ee.Algorithms.CannyEdgeDetector(image=forest_mask, threshold=0.5, sigma=1)
    edge_density = forest_edges.reduceNeighborhood(
        ee.Reducer.sum(), ee.Kernel.circle(radius=radius_m, units='meters')
    ).rename(f'forest_edge_{radius_m}')
    kernel = ee.Kernel.circle(radius=radius_m, units='meters', normalize=True)
    non_forest_buffer = non_forest_mask.focal_max(radius=100, units='meters')
    forest_core = forest_mask.And(non_forest_buffer.Not())
    pct = forest_core.reduceNeighborhood(ee.Reducer.mean(), kernel).multiply(100).rename(f'forest_core_pct_{radius_m}')
    return ee.Image.cat([edge_density, pct])


def export_feature_rasters_two_periods(feature, year=2025, asset_folder='biodiversity'):
    try:
        aoi_buffered = feature.buffer(5000).bounds()
        # Define month groups
        winter_fall_months = [1, 2, 3, 10, 11, 12]
        summer_months = [4, 5, 6, 7, 8, 9]

        # Get mode images for both periods
        winter_fall_img = get_dw_mode_image(winter_fall_months, year).clip(aoi_buffered)
        summer_img = get_dw_mode_image(summer_months, year).clip(aoi_buffered)

        # Compute metrics for each period
        def build_composite(dw_img):
            cover_100m = compute_dw_percent_cover(dw_img, 100)
            cover_10km = compute_dw_percent_cover(dw_img, 10000)
            forest_metrics_100m = compute_forest_metrics(dw_img, 100)
            forest_metrics_10km = compute_forest_metrics(dw_img, 10000)
            return ee.Image.cat([cover_100m, cover_10km, forest_metrics_100m, forest_metrics_10km])

        winter_fall_composite = build_composite(winter_fall_img)
        summer_composite = build_composite(summer_img)

        # Export both composites
        for period_name, img in [("winter_fall", winter_fall_composite), ("summer", summer_composite)]:
            export_desc = f"{period_name}_{year}"

            task = ee.batch.Export.image.toDrive(
                image=img,
                description=export_desc,
                folder=asset_folder,
                fileNamePrefix=export_desc,
                region=feature,
                scale=100,
                maxPixels=1e13
            )
            task.start()
            print(f"✔ Export started for {period_name} composite")
    except Exception as e:
        print(f"❌ Export failed: {e}")
        raise

In [58]:
aoifc = gdf_to_ee_geometry(aoi_gdf)
export_feature_rasters_two_periods(aoifc)

✔ Export started for winter_fall composite
✔ Export started for summer composite
